In [1]:
!pip install --upgrade pip
!pip install --upgrade gradio fastapi uvicorn

  Using cached fastapi-0.114.0-py3-none-any.whl.metadata (27 kB)


In [2]:
import gradio as gr 
import pickle

In [3]:
with open('Final_model_xgb.pkl','rb') as file:
    model = pickle.load(file)

In [4]:
model.feature_names_in_

array(['lead_time', 'market_segment_type', 'no_of_special_requests',
       'avg_price_per_room', 'no_of_adults', 'no_of_weekend_nights',
       'required_car_parking_space', 'no_of_week_nights', 'arrival_day',
       'arrival_month', 'arrival_weekday'], dtype='<U26')

In [5]:
model.predict([[10,1,0,100,2,2,0,1,4,5,2]])[0]

0

In [6]:
def prediction(lt,mst,spcl,price,adul,wkend,park,wk,ar_d,ar_m,ar_w):
    input_data = [[lt,mst,spcl,price,adul,wkend,park,wk,ar_d,ar_m,ar_w]]
    
    pred=model.predict_proba(input_data)[:,1][0]
    
    if pred>0.5:
        return f'This Booking is more likely to get cancelled: Chances={round(pred*100,2)}%'
    else:
        return f'This Booking is less likely to get cancelled: Chances={round(pred*100,2)}%'

In [7]:
prediction(10,1,1,100,2,2,0,1,4,5,3)

'This Booking is less likely to get cancelled: Chances=13.66%'

In [8]:
iface=gr.Interface(fn=prediction,
                  inputs=[gr.Number(label='How many Prior days booking was made?'),
                          gr.Dropdown([('Online',1),('Offline',0)],label='How the booking was made?'),
                          gr.Dropdown([0,1,2,3,4,5],label='How many special requests made?'),
                          gr.Number(label='What is the price per room offered?'),
                          gr.Dropdown([1,2,3,4],label='How many adults per room?'),
                          gr.Number(label='How many weekend nights in the stay?'),
                          gr.Dropdown([('Yes',1),('No',0)],label='Does booking includs Parking facility?'),
                          gr.Number(label='How many week nights in the stay?'),
                          gr.Slider(minimum=1,maximum=31,step=1,label='What is the day of arrival?'),
                          gr.Slider(minimum=1,maximum=12,step=1,label='What is the month of arrival?'),
                          gr.Dropdown([('Mon',0),('Tue',1),('Wed',2),('Thus',3),('Fri',4),('Sat',5),('Sun',6)],
                                     label='What is the weekend of arrival?')],
                  
                  outputs=gr.Textbox(label='Prediction'),
                  title='INN Group of Hotels',
                  description='This application will forcast the calcellation of booking',
                  allow_flagging='never')


In [9]:
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [2]:
import streamlit as st 
import numpy as np 
import pandas as pd  
import pickle 
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

with open('Final_model_xgb.pkl','rb')as file:
    model = pickle.load(file)

#input_data = [[lt,mst,spcl,price,adul,wkend,park,wk,ar_d,ar_m,ar_w]]
def prediction(input_data):
    pred=model.predict_proba(input_data)[:,1][0]
    
    if pred>0.5:
        return f'This Booking is more likely to get cancelled: Chances={round(pred*100,2)}%'
    else:
        return f'This Booking is less likely to get cancelled: Chances={round(pred*100,2)}%'

def main():
    st.image('hotelimage.png',use_column_width=True)
    st.title('INN Hotels')
    lt=st.text_input('Enter Lead time')
    mkt=(lambda x : 1 if x =='Online' else 0)(st.selectbox('Enter the type of booking',['Online','Offline']))
    spcl=st.selectbox('How many special requests have been made?',[0,1,2,3,4,5])
    price=st.text_input('Enter the price of the room.')
    adults= st.selectbox('How many Adults per room?',[1,2,3,4])
    wknd=st.text_input('How many weekend nights?')
    prk=(lambda x : 1 if x=='Yes' else 0)(st.selectbox('Does booking includes parking facility.',['Yes','No']))
    wk=st.text_input('How many weekday nights')
    arr_d=st.slider('What will be the day of arrival.',min_value=1,max_value=31,step=1)
    arr_mon=st.slider('What will be the month of arrival.',min_value=1,max_value=12,step=1)
    week_lambda= (lambda x : 0 if x=='Mon' else 1 if x=='Tue' else 2 if x=='Wed' else 3 
                  if x=='Thus' else 4 if x=='Fri' else 5 if x=='Sat' else 6 )
    arr_wd= week_lambda(st.selectbox('What is the Weekday of arrival?.',['Mon','Tue','Wed','Thus','Fri','Sat','Sun']))

    input_data = [[lt,mkt,spcl,price,adults,wknd,prk,wk,arr_d,arr_mon,arr_wd]]


    if st.button('Predict'):
        responce=prediction(input_data)
        st.success(responce)


if __name__ =='__main__':
    main()

2024-09-25 19:28:36.731 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:28:36.759 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:28:36.842 
  command:

    streamlit run /Users/devanshu29bansalgmail.com/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-09-25 19:28:36.843 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:28:36.843 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:28:36.843 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:28:36.843 Thread 'MainThread': missing ScriptRunContext! This war

2024-09-25 19:28:36.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 19:28:36.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
